In [8]:
import pandas as pd
import re, os
import numpy as np
import string
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import preprocessor as p
from IPython.display import clear_output
os.chdir("D:/Semester 8/.Final Project/.machine learning")

In [32]:
data_train = pd.read_csv(r"D:\Semester 8\.Final Project\.machine learning\dataset\cyberbullying.csv")
data_train.drop(columns='Id', inplace=True)
data_train.columns = ['Sentiment', 'Text']

,Sentiment,Text
0,negative,<USERNAME> TOLOL!! Gak ada hubungan nya kegug...
1,negative,Geblek lo tata...cowo bgt dibela2in balikan......
2,negative,Kmrn termewek2 skr lengket lg duhhh kok labil ...
3,negative,"Intinya kalau kesel dengan ATT nya, gausah ke ..."
4,negative,"hadewwwww permpuan itu lg!!!!sakit jiwa,knp ha..."
...,...,...
395,positive,Bangga sama suami yg selalu ingat istri disela...
396,positive,Apaoun pekerjaannya yg penting halal u tuk men...
397,positive,Gojek itu mayoritas pegangguran yang lama gak ...
398,positive,<USERNAME> aslinya cantik dan ayu loh mbak kr...


In [33]:
from sklearn import preprocessing
lb = preprocessing.LabelBinarizer()
data_train['Sentiment'] = lb.fit_transform(data_train['Sentiment'])
data_train


,Sentiment,Text
0,0,<USERNAME> TOLOL!! Gak ada hubungan nya kegug...
1,0,Geblek lo tata...cowo bgt dibela2in balikan......
2,0,Kmrn termewek2 skr lengket lg duhhh kok labil ...
3,0,"Intinya kalau kesel dengan ATT nya, gausah ke ..."
4,0,"hadewwwww permpuan itu lg!!!!sakit jiwa,knp ha..."
...,...,...
395,1,Bangga sama suami yg selalu ingat istri disela...
396,1,Apaoun pekerjaannya yg penting halal u tuk men...
397,1,Gojek itu mayoritas pegangguran yang lama gak ...
398,1,<USERNAME> aslinya cantik dan ayu loh mbak kr...


In [34]:
# Replace Acronym

acronym = {}
with open(r'D:\Semester 8\.Final Project\.machine learning\dataset\acronym.txt') as file:
    # next(file) # skip the first line "before, after"
    for line in file:
        k, v = line.strip().split(' = ')
        acronym[k] = v

def replace_acronym(tweet):
    return ' '.join(acronym.get(x.upper(), x) for x in tweet.split())

In [35]:
# Replace Slang Words

slang = {}
with open(r'D:\Semester 8\.Final Project\.machine learning\dataset\kamusalay.csv') as file:
    # next(file) # skip the first line "before, after"
    for line in file:
        k, v = line.strip().split(',')
        slang[k] = v

def replace_slang(tweet):
    return ' '.join(slang.get(x.lower(), x) for x in tweet.split())

In [36]:
# Replace Emoji

from emoji import UNICODE_EMOJI

emoji = {}
with open(r'D:\Semester 8\.Final Project\.machine learning\dataset\emoji.txt', encoding="utf8") as file:
    # next(file) # skip the first line "before, after"
    for line in file:
        k, v = line.strip().split(', ')
        emoji[k] = v

def is_emoji(s):
    return s in UNICODE_EMOJI

def replace_emoji(tweet):
    result = ' '
    for char in tweet:
        if is_emoji(char):
            result = result + ' ' + emoji.get(char, char)
        else:
            result += emoji.get(char, char)
    return result

In [ ]:
teks.replace('<USERNAME>', '')

In [43]:
# get stopwords
factory = StopWordRemoverFactory()
stopword = factory.create_stop_word_remover()

# initiate stemmer
factory = StemmerFactory()
stemmer = factory.create_stemmer()

# tweet-preprocess
p.set_options(p.OPT.URL, p.OPT.MENTION, p.OPT.HASHTAG, p.OPT.NUMBER)

def data_cleaning(data): 

    cek = []
    for i, text in enumerate (data['Text']):
        text = text.lower()
        # stopword removal
        text = stopword.remove(text)
        text = re.sub("\d+", "", text)
        
        # remove @username
        text = p.clean(text)
        text = re.sub("(@[^\s]+|@[A-Za-z0-9]+)", " ", text)

        # replace emoji
        text = replace_emoji(str(text))

        # replace slang words
        text = replace_slang(str(text))        

        # replace abbreviation
        text = replace_acronym(str(text))        

        # stemming
        text = stemmer.stem(text)
        
        # remove duplicate char
        text = re.sub(r'(.)\1+$', r'\1', text)
        # tokenizing + append
        cek.append(text.strip())

    data['Clean text'] = cek

    return data

In [44]:
clean_data = data_cleaning(data_train)
clean_data

,Sentiment,Text,Clean text
0,0,<USERNAME> TOLOL!! Gak ada hubungan nya kegug...,username tolol tidak hubung nya gugur dengan p...
1,0,Geblek lo tata...cowo bgt dibela2in balikan......,geblek kamu tata cowo banget bela balik hadeww...
2,0,Kmrn termewek2 skr lengket lg duhhh kok labil ...,kemarin mewek sekarang lengket lagi duhhh kok ...
3,0,"Intinya kalau kesel dengan ATT nya, gausah ke ...",inti kalau kesal att nya tidak usah anak juga ...
4,0,"hadewwwww permpuan itu lg!!!!sakit jiwa,knp ha...",hadewwwww permpuan lg sakit jiwa knp daerah is...
...,...,...,...
395,1,Bangga sama suami yg selalu ingat istri disela...,bangga sama suami yang selalu ingat istri sela...
396,1,Apaoun pekerjaannya yg penting halal u tuk men...,apa kerja yang penting halal kamu untuk nafkah...
397,1,Gojek itu mayoritas pegangguran yang lama gak ...,gojek mayoritas pegangguran lama tidak kerja t...
398,1,<USERNAME> aslinya cantik dan ayu loh mbak kr...,username asli cantik ayu lah mbak karena aku t...
